**Question 4**

Run the below cell to mount the google where you might have stored the dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Run the below cell to install wandb

In [2]:
!pip install wandb

     |████████████████████████████████| 1.8MB 26.7MB/s 
     |████████████████████████████████| 174kB 46.5MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 133kB 50.5MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=ae64eb752d81d24e72e735da6b3057a924be04535b8f96a74def056e22a151d6
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=91555566649d5b9395293041144c823de583b9b9c0cefdfdb085f4ad009b1c36
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


Run the below cell to initialize the RNN class. It is necessary step since we will be using few functions of RNN class


*   Do not forget to upload the config file present in the github before running the cell




In [4]:
import numpy as np
import tensorflow as tf
import wandb
import config
import random
from tensorflow import keras

import ipywidgets as widgets

import numpy as np
import tensorflow as tf
import wandb
import config
import random
from tensorflow import keras
from tensorflow.keras.utils import plot_model

configuration = config.config
model_folder ='./model'

train_text =[]
val_text =[]

train_target_text =[]
val_target_text = []

train_inp = set()
val_inp = set()

train_tar = set()
val_tar = set()



with open(configuration.get("train_path"),'r') as f:
  lines = f.read().split("\n")
  for line in lines:
    try:
       target_text,input_text, _ = line.split("\t")
    except:
      continue
    target_text = "\t" + target_text + "\n"

    train_text.append(input_text)
    train_target_text.append(target_text)

    for char in input_text:
        if char not in train_inp:
            train_inp.add(char)
    for char in target_text:
        if char not in train_tar:
            train_tar.add(char)


with open(configuration.get("val_path"),'r') as f:
  lines = f.read().split("\n")
  
  for line in lines:
    try:
      target_text, input_text, _ = line.split("\t")
    except:
      continue
    #input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"

    val_text.append(input_text)
    val_target_text.append(target_text)

    for char in input_text:
        if char not in val_inp:
            val_inp.add(char)
    for char in target_text:
        if char not in val_tar:
            val_tar.add(char)






class RNN():
  def __init__(self):
    


    self.epoch = wandb.config.epoch
    self.batch_size = wandb.config.batch_size
    self.num_encoder_tokens = len(train_inp)+1
    self.num_decoder_tokens = len(train_tar)
    self.max_encoder_seq_length = max([len(txt) for txt in train_text])
    self.max_decoder_seq_length = max([len(txt) for txt in train_target_text])
    #self.input_token_index, self.target_token_index = self.map_characters()
    self.input_token_index = {'j': 0, 'q': 1, 'i': 2, 'b': 3, 'm': 4, 'a': 5, 'o': 6, 'z': 7, 'u': 8, 'r': 9, 'd': 10, 'n': 11, 'c': 12, 'k': 13, 'l': 14, 'x': 15, 'v': 16, 'g': 17, 'w': 18, 'y': 19, 'p': 20, 'f': 21, 's': 22, 'h': 23, 't': 24, 'e': 25, ' ': 26}
    self.target_token_index = {'ढ': 0, 'ब': 1, 'ष': 2, 'ू': 3, '़': 4, 'ऊ': 5, 'आ': 6, 'ग': 7, 'ख': 8, 'ृ': 9, 'अ': 10, 'ु': 11, 'त': 12, 'ह': 13, 'ऑ': 14, 'ऐ': 15, 'घ': 16, 'ि': 17, 'द': 18, 'च': 19, 'ट': 20, 'ॉ': 21, 'ं': 22, 'म': 23, 'ओ': 24, 'ङ': 25, 'ई': 26, 'उ': 27, 'ल': 28, 'इ': 29, 'झ': 30, 'ॐ': 31, 'छ': 32, 'श': 33, 'औ': 34, 'भ': 35, 'क': 36, 'ड': 37, 'ज': 38, 'ा': 39, 'ः': 40, 'न': 41, 'र': 42, 'य': 43, 'फ': 44, 'ौ': 45, 'ए': 46, 'ऋ': 47, 'ो': 48, 'ै': 49, 'े': 50, 'ध': 51, '्': 52, 'ञ': 53, 'थ': 54, 'ँ': 55, 'ण': 56, 'व': 57, 'प': 58, 'ॅ': 59, 'ठ': 60, '\t': 61, 'स': 62, '\n': 63, 'ी': 64}


    self.train = self.encoding(train_text, train_target_text)
    self.validation = self.encoding(val_text, val_target_text)
    self.embedding_size =wandb.config.embedding_size
    self.celltype=wandb.config.celltype
    self.number_encoder_layer=wandb.config.num_encoder
    self.number_decoder_layer=wandb.config.num_decoder
    self.hidden_layers =wandb.config.hidden_layer_size
    self.lr = 0.001
    #self.beam_search=wandb.config.epoch
    self.beam_size=wandb.config.beam_size
    self.dr=wandb.config.dropout

    self.reverse_token_index, self.reverse_target_index = self.reverse_character_maps()

    self.tab_index=self.target_token_index['\t']

    print(self.tab_index,self.target_token_index['\n'])

  def map_characters(self):
    input_token_index = dict([(char, i) for i, char in enumerate(train_inp)])
    target_token_index = dict([(char, i) for i, char in enumerate(train_tar)])
    input_token_index[" "] = len(input_token_index)
    print(input_token_index)
    print(target_token_index)
    return input_token_index, target_token_index

  def reverse_character_maps(self):
    reverse_token_index = dict([( i,char) for i, char in enumerate(train_inp)])
    reverse_target_index = dict([( i,char) for i, char in enumerate(train_tar)])

    return reverse_token_index, reverse_target_index


  def encoding(self,text, target_text):
    decoder_target_data = np.zeros((len(text), self.max_decoder_seq_length, self.num_decoder_tokens), dtype="float32")
    encoder_input_data = np.zeros((len(text), self.max_encoder_seq_length), dtype="float32")
    decoder_input_data = np.zeros((len(text), self.max_decoder_seq_length), dtype="float32")

    for i, (input, target) in enumerate(zip(text, target_text)):
      for t, char in enumerate(input):
          encoder_input_data[i, t] = self.input_token_index[char]
      encoder_input_data[i, t + 1 :] =  self.input_token_index[" "]
      for t, char in enumerate(target):
          decoder_input_data[i, t] = self.target_token_index[char]
          if t > 0:
              decoder_target_data[i, t - 1, self.target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :] =  self.target_token_index["\n"]
      decoder_target_data[i, t:, self.target_token_index["\n"]] = 1.0

    return decoder_target_data, encoder_input_data, decoder_input_data
    



  def loss(self,x1,x2,y_true,training):
    y_predict = self.model([x1,x2],training)
    return y_predict,self.cce(y_true,y_predict)

  def grad(self, input1,input2, targets,training):
    with tf.GradientTape() as tape:
      logits,loss_value = self.loss( input1,input2, targets, training)
    return logits,loss_value, tape.gradient(loss_value, self.model.trainable_variables)

  def optimize(self,loss_tensor):
    opt = tf.keras.optimizers.Adam(learning_rate=self.lr)
    #opt = tf.keras.optimizers.RMSprop(learning_rate=self.lr)
    train_op = opt.minimize(loss_tensor)

    return train_op

  def fill_feed_dict(self,dataset):
    target_output_data= dataset[0]
    train_data = dataset[1]
    target_data = dataset[2]

    start = [random.randint(0,train_data.shape[0]-1) for i in range(self.batch_size) ]

    encoder_input_data = train_data[start,:]
    decoder_input_data = target_data[start,:]
    decoder_output_data = target_output_data[start,:,:]


    return encoder_input_data, decoder_input_data,decoder_output_data,start

  def greedy_search(self,predictions):
    return tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

  def get_accuracy(self,prediction,start,type_):

    #result_text = []
    count =0
    print(type_)
    for i in range(prediction.shape[0]):
      string=''
      for j in range(prediction.shape[1]):
        if ((prediction[i,j] ==self.target_token_index['\n']) or (prediction[i,j] ==self.tab_index)):
          break;
          
        else:
          string = string + self.reverse_target_index[prediction[i,j]]
          
      if(type_=="train"):
        print(string, train_target_text[start[i]][1:-1])
        if(string == train_target_text[start[i]][1:-1]):
          print(string, train_target_text[start[i]][1:-1])
          count +=1
      elif(type_=="val"):
        print(string, val_target_text[start[i]][1:-1])
        if(string == val_target_text[start[i]][1:-1]):
          count +=1
      

      



    return count/prediction.shape[0]


  def beam_search_function(self,predictions):

    result,indices = tf.nn.top_k(predictions, self.beam_size)
    log_result = tf.math.log(result)

    y_pred = np.zeros((log_result.shape[0],self.max_decoder_seq_length ))

    for i in range(result.shape[0]):
      for j in range(result.shape[1]):
        if(j==0):
          pred = np.max(log_result[i,j,:].numpy())
          y_pred[i,j] = int(indices.numpy()[i,j,np.argmax(log_result[i,j,:].numpy()) ])
        else:
          pred_ = pred + log_result[i,j,:].numpy()
          pred = np.max(pred_)
          y_pred[i,j] =  int(indices.numpy()[i,j, np.argmax(pred_)] )
    return y_pred


  def rnn_model(self, e_input, d_input):

    encoder = tf.keras.layers.SimpleRNN(self.hidden_layers,return_sequences=True, return_state=True,dropout = self.dr)

    output, state = encoder(e_input)

    for i in range(self.number_encoder_layer-1):

      encoder = tf.keras.layers.SimpleRNN(self.hidden_layers,return_sequences=True, return_state=True,dropout = self.dr)

      output, state = encoder(output)
   
    decoder = tf.keras.layers.SimpleRNN(self.hidden_layers,return_sequences=True, return_state=True)

    d_output, d_state = decoder(d_input,initial_state=[state])

    for i in range(self.number_decoder_layer-1):

      decoder = tf.keras.layers.SimpleRNN(self.hidden_layers,return_sequences=True, return_state=True)

      d_output, d_state = decoder(d_output,initial_state=[state])

    decoder_dense = tf.keras.layers.Dense(self.num_decoder_tokens, activation="softmax")

    decoder_outputs = decoder_dense(d_output)

    return decoder_outputs, state



  def gru_model(self,e_input, d_input):

    encoder = tf.keras.layers.GRU(self.hidden_layers,return_sequences=True, return_state=True,dropout = self.dr)

    output, state = encoder(e_input)

    for i in range(self.number_encoder_layer-1):

      encoder = tf.keras.layers.GRU(self.hidden_layers,return_sequences=True, return_state=True,dropout = self.dr)

      output, state = encoder(output)
   
    decoder = tf.keras.layers.GRU(self.hidden_layers,return_sequences=True, return_state=True)

    d_output, d_state = decoder(d_input,initial_state=[state])

    for i in range(self.number_decoder_layer-1):

      decoder = tf.keras.layers.GRU(self.hidden_layers,return_sequences=True, return_state=True)

      d_output, d_state = decoder(d_output,initial_state=[state])

    decoder_dense = tf.keras.layers.Dense(self.num_decoder_tokens, activation="softmax")

    decoder_outputs = decoder_dense(d_output)

    return decoder_outputs,state

  def lstm_model(self,e_input, d_input):

    encoder = tf.keras.layers.LSTM(self.hidden_layers,return_sequences=True, return_state=True,dropout = self.dr)

    output, state,c = encoder(e_input)

    for i in range(self.number_encoder_layer-1):

      encoder = tf.keras.layers.LSTM(self.hidden_layers,return_sequences=True, return_state=True,dropout = self.dr)

      output, state,c = encoder(output)
   
    decoder = tf.keras.layers.LSTM(self.hidden_layers,return_sequences=True, return_state=True)

    d_output, d_state, d_c = decoder(d_input,initial_state=[state,c])

    for i in range(self.number_decoder_layer-1):

      decoder = tf.keras.layers.LSTM(self.hidden_layers,return_sequences=True, return_state=True)

      d_output, d_state, d_c = decoder(d_output,initial_state=[state, c])

    decoder_dense = tf.keras.layers.Dense(self.num_decoder_tokens, activation="softmax")

    decoder_outputs = decoder_dense(d_output)

    return decoder_outputs, [state,c]


  def save_model(self):
    self.model.save("model2")



  def run_training(self):
    with tf.device('/device:GPU:0'):
      
      input = keras.Input(shape=(self.max_encoder_seq_length),batch_size=self.batch_size)

      d_input = keras.Input(shape=(self.max_decoder_seq_length),batch_size=self.batch_size)

      e_embedding_output =tf.keras.layers.Embedding(input_dim=self.num_encoder_tokens, output_dim=self.embedding_size,input_length=self.max_encoder_seq_length)(input)

      d_embedding_output =tf.keras.layers.Embedding(self.num_decoder_tokens, self.embedding_size, input_length=self.max_decoder_seq_length)(d_input)



      if(self.celltype =="rnn"):

        decoder_outputs,state =self.rnn_model(e_embedding_output,d_embedding_output)

      elif(self.celltype =="gru"): 

        decoder_outputs,state =self.gru_model(e_embedding_output,d_embedding_output)


      elif(self.celltype =="lstm"): 

        decoder_outputs,state =self.lstm_model(e_embedding_output,d_embedding_output)


      

      self.model = tf.keras.Model([input, d_input], decoder_outputs)
      print(self.model.layers)

      self.cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
      self.opt = tf.keras.optimizers.Adam(learning_rate=self.lr)
      self.epoch_loss_avg = tf.keras.metrics.Mean()
      self.train_metric=train_acc_metric = keras.metrics.CategoricalAccuracy()
      self.val_metric=train_acc_metric = keras.metrics.CategoricalAccuracy()
      plot_model(self.model, to_file='model.png')



      for epoch in range(self.epoch):
        prediction_decoder_input = np.zeros(self.train[2].shape)

        prediction_decoder_input[:,0] = self.target_token_index["\t"] 

        val_prediction_decoder_input = np.zeros(self.validation[2].shape)

        val_prediction_decoder_input[:,0] = self.target_token_index["\t"]

        for step in range(int(self.train[1].shape[0]/self.batch_size)):

            encoder_input_data, decoder_input_data,decoder_output_data,start  = self.fill_feed_dict(self.train)
            logits, loss_value, grads = self.grad(encoder_input_data, decoder_input_data,decoder_output_data,True)
            self.opt.apply_gradients(zip(grads, self.model.trainable_variables))

            self.epoch_loss_avg.update_state(loss_value)

            self.train_metric.update_state(decoder_output_data, logits)
        
            
            #predictions = self.model([self.train[1][:int(self.train[1].shape[0]/2)],prediction_decoder_input[:int(self.train[1].shape[0]/2)]],training=True)

        if( self.beam_size == 1):
            y_pred = self.greedy_search(logits)
            accuracy = self.get_accuracy(y_pred.numpy(),start,"train")
        else:
            y_pred = self.beam_search_function(logits)
            accuracy = self.get_accuracy(y_pred,start,"train")


        for step in range(int(self.validation[1].shape[0]/self.batch_size)):

          encoder_input_data, decoder_input_data,decoder_output_data ,start = self.fill_feed_dict(self.validation)

          val_predictions = self.model([encoder_input_data,decoder_input_data],training=False)

          self.val_metric.update_state(decoder_output_data, val_predictions) 

          #val_predictions = self.model([self.validation[1],val_prediction_decoder_input],training=False)
        val_loss = self.cce(decoder_output_data,val_predictions)

        if(self.beam_size == 1):
            y_pred = self.greedy_search(val_predictions)
            val_accuracy = self.get_accuracy(y_pred.numpy(),start,"val")
        else:
            y_pred = self.beam_search_function(val_predictions)
            val_accuracy = self.get_accuracy(y_pred,start,"val")

        print("cross_entropy_accuracy",self.train_metric.result().numpy())

        print("cross_entropy_val_accuracy",self.val_metric.result().numpy())


        print("epoch:  ",epoch, "   accuracy :  ", accuracy, "    loss: ",self.epoch_loss_avg.result().numpy(),"   val_accuracy :  ", val_accuracy, "  val_loss: ",val_loss.numpy())

        wandb.log({
        "epoch": epoch,
        "loss": self.epoch_loss_avg.result().numpy(),
        "accuracy": accuracy,
        "categorical_accuracy":self.train_metric.result().numpy(),
        "val_loss": val_loss.numpy(),
        "val_accuracy": val_accuracy,
        "categorical_val_accuracy":self.val_metric.result().numpy()})




def begin():
  wandb.init(project ='assignment3-part1',config=configuration, magic=True,reinit = True)    
  wandb.run.name = wandb.run.name = 'hl-'+str(wandb.config.hidden_layer_size)+'-ne-'+str(wandb.config.num_encoder)+'-nd-'+str(wandb.config.num_encoder)+'-es-'+str(wandb.config.embedding_size)+'-ct-'+str(wandb.config.celltype)+'-d-'+str(wandb.config.dropout)+'-bs-'+str(wandb.config.batch_size)+'-e-'+str(wandb.config.epoch)

  
  #rnn=RNN()
  #rnn.run_training()
  #rnn.save_model()

if __name__ == "__main__":
  #begin()
  rnn=RNN()
    

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


61 63





*   Upload the "seq2seqmodel.tar.gz" file from the github page
*   Run the below cell





In [6]:
!tar -xvf seq2seqmodel.tar.gz

model/
model/keras_metadata.pb
model/saved_model.pb
model/assets/
model/variables/
model/variables/variables.data-00000-of-00001
model/variables/variables.index


In [ ]:
{'j': 0, 'q': 1, 'i': 2, 'b': 3, 'm': 4, 'a': 5, 'o': 6, 'z': 7, 'u': 8, 'r': 9, 'd': 10, 'n': 11, 'c': 12, 'k': 13, 'l': 14, 'x': 15, 'v': 16, 'g': 17, 'w': 18, 'y': 19, 'p': 20, 'f': 21, 's': 22, 'h': 23, 't': 24, 'e': 25, ' ': 26}
{'ढ': 0, 'ब': 1, 'ष': 2, 'ू': 3, '़': 4, 'ऊ': 5, 'आ': 6, 'ग': 7, 'ख': 8, 'ृ': 9, 'अ': 10, 'ु': 11, 'त': 12, 'ह': 13, 'ऑ': 14, 'ऐ': 15, 'घ': 16, 'ि': 17, 'द': 18, 'च': 19, 'ट': 20, 'ॉ': 21, 'ं': 22, 'म': 23, 'ओ': 24, 'ङ': 25, 'ई': 26, 'उ': 27, 'ल': 28, 'इ': 29, 'झ': 30, 'ॐ': 31, 'छ': 32, 'श': 33, 'औ': 34, 'भ': 35, 'क': 36, 'ड': 37, 'ज': 38, 'ा': 39, 'ः': 40, 'न': 41, 'र': 42, 'य': 43, 'फ': 44, 'ौ': 45, 'ए': 46, 'ऋ': 47, 'ो': 48, 'ै': 49, 'े': 50, 'ध': 51, '्': 52, 'ञ': 53, 'थ': 54, 'ँ': 55, 'ण': 56, 'व': 57, 'प': 58, 'ॅ': 59, 'ठ': 60, '\t': 61, 'स': 62, '\n': 63, 'ी': 64}


Run the below cell to compile the model

In [8]:
import numpy as np
import tensorflow as tf
#import wandb
import config
import random
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import keras.backend as K
rnn = RNN()
input_token_index = {'j': 0, 'q': 1, 'i': 2, 'b': 3, 'm': 4, 'a': 5, 'o': 6, 'z': 7, 'u': 8, 'r': 9, 'd': 10, 'n': 11, 'c': 12, 'k': 13, 'l': 14, 'x': 15, 'v': 16, 'g': 17, 'w': 18, 'y': 19, 'p': 20, 'f': 21, 's': 22, 'h': 23, 't': 24, 'e': 25, ' ': 26}
target_token_index = {'ढ': 0, 'ब': 1, 'ष': 2, 'ू': 3, '़': 4, 'ऊ': 5, 'आ': 6, 'ग': 7, 'ख': 8, 'ृ': 9, 'अ': 10, 'ु': 11, 'त': 12, 'ह': 13, 'ऑ': 14, 'ऐ': 15, 'घ': 16, 'ि': 17, 'द': 18, 'च': 19, 'ट': 20, 'ॉ': 21, 'ं': 22, 'म': 23, 'ओ': 24, 'ङ': 25, 'ई': 26, 'उ': 27, 'ल': 28, 'इ': 29, 'झ': 30, 'ॐ': 31, 'छ': 32, 'श': 33, 'औ': 34, 'भ': 35, 'क': 36, 'ड': 37, 'ज': 38, 'ा': 39, 'ः': 40, 'न': 41, 'र': 42, 'य': 43, 'फ': 44, 'ौ': 45, 'ए': 46, 'ऋ': 47, 'ो': 48, 'ै': 49, 'े': 50, 'ध': 51, '्': 52, 'ञ': 53, 'थ': 54, 'ँ': 55, 'ण': 56, 'व': 57, 'प': 58, 'ॅ': 59, 'ठ': 60, '\t': 61, 'स': 62, '\n': 63, 'ी': 64}


reverse_token_index = dict([( i,char) for char,i in input_token_index.items()])
reverse_target_index = dict([( i,char) for char,i in target_token_index.items()])

model_path = "model"

model = tf.keras.models.load_model(model_path)

encoder_input = model.input[0]
decoder_input = model.input[1]


print(encoder_input.shape)
print(decoder_input.shape)

encoder_embedding = model.layers[1]
decoder_embedding = model.layers[4]

encoder_1 = model.layers[3]
encoder_2 = model.layers[5]


decoder_1 = model.layers[6]

dense = model.layers[7]


print(encoder_embedding(encoder_input).shape)
o=encoder_embedding(encoder_input)
o1,_,_=encoder_1(o)
o2,_,_=encoder_2(o1)




#attn_func = K.function(inputs = [model.get_input_at(0), K.learning_phase()],outputs = [encoder_3.output])

encoder_output, encoder_output_state1, encoder_output_state2= encoder_2.output
encoder_output_state= [encoder_output_state1,encoder_output_state2]
print(encoder_output_state1.shape)
encoder_model = tf.keras.Model(encoder_input, outputs=[encoder_output_state,o2])

print(decoder_embedding(decoder_input).shape)
decoder_state_input1 = keras.Input(shape=(config.config.get("hidden_layer_size"),), name="input_3")
decoder_state_input2 = keras.Input(shape=(config.config.get("hidden_layer_size"),), name="input_5")
decoder_state_input = [decoder_state_input1,decoder_state_input2]
#print(decoder_state_input.shape)
decoder_output, state,state2 = decoder_1(decoder_embedding(decoder_input), initial_state=decoder_state_input)

#decoder_state = keras.Input(shape=(config.config.get("hidden_layer_size"),), name="input_3")

dense_outputs = dense(decoder_output)


decoder_model = tf.keras.Model(inputs=[decoder_input,decoder_state_input], outputs =[dense_outputs,state,state2])






def decode_sequence(input_seq):


    states_value,_ = encoder_model.predict(input_seq)


    target_seq = np.zeros((input_seq.shape[0], 1))

    target_seq[:, 0] = target_token_index["\t"]


    stop_condition = False
    decoded_sentence = ""
    pred = 1
    while not stop_condition:
        #print(states_value.shape)
        output_tokens, h,c = decoder_model.predict([target_seq,states_value])
        #print("xyz")

        tokens = (-output_tokens[0, -1, :]).argsort()[:rnn.beam_size]
        temp = output_tokens[0, -1, :][tokens]
        temp = temp*pred

        #print(temp,len(temp))

        pred_index = np.argmax(temp)
        sampled_token_index = tokens[pred_index]

        pred = temp[pred_index]
        
        #sampled_token_index = np.argmax(output_tokens[0, -1, :])
        #print(sampled_token_index)
        sampled_char = reverse_target_index[sampled_token_index]
        decoded_sentence += sampled_char


        if sampled_char == "\n" or len(decoded_sentence) > 22:#rnn.max_decoder_seq_length:
            stop_condition = True

        #print(decoded_sentence)

        target_seq = np.zeros((input_seq.shape[0], 1))
        target_seq[:, 0] = sampled_token_index


        states_value = [h,c]
    return decoded_sentence





61 63
(64, 20)
(64, 21)
(64, 20, 16)
(64, 128)
(64, 21, 16)


The below cell prepares the test data

In [9]:

test_text =[]

test_target_text =[]

test_inp = set()

test_tar = set()

with open(configuration.get("test_path"),'r') as f:
  lines = f.read().split("\n")
  
  for line in lines:
    try:
       target_text,input_text, _ = line.split("\t")
    except:
      continue
    #input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"

    test_text.append(input_text)
    test_target_text.append(target_text)

    for char in input_text:
        if char not in test_inp:
            test_inp.add(char)
    for char in target_text:
        if char not in test_tar:
            test_tar.add(char)

Below is a helper function

In [10]:
def encoding(text, target_text):
  decoder_target_data = np.zeros((len(text), 21,65 ), dtype="float32")
  encoder_input_data = np.zeros((len(text), 20), dtype="float32")
  decoder_input_data = np.zeros((len(text), 21), dtype="float32")

  for i, (input, target) in enumerate(zip(text, target_text)):
    for t, char in enumerate(input):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] =  input_token_index[" "]
    for t, char in enumerate(target):
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] =  target_token_index["\n"]
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return decoder_target_data, encoder_input_data, decoder_input_data

**Question 4 (a)(b)**

Run the below cell for part a and b of question 5

In [ ]:

import csv

truth_value=[]
predicted_value =[]

with open('test_value.csv', mode='w') as file:
  fieldnames = ['text', 'truth value', 'predicted value']
  writer = csv.DictWriter(file, fieldnames=fieldnames)
  writer.writeheader()
  decoder_target_data, encoder_input_data, decoder_input_data=encoding(test_text, test_target_text)
  #print(encoder_input_data))
  count =0
  with tf.device('/device:GPU:0'):
    for seq_index in range(len(test_text)):

        input_seq = encoder_input_data[seq_index : seq_index +1]
        input_seq=np.tile(input_seq ,(64,1))

        
        decoded_sentence = decode_sequence(input_seq)

        if(test_target_text[seq_index][1:-1] == decoded_sentence[:-1]):
            count +=1
        truth_value.append(test_target_text[seq_index][1:-1])
        predicted_value.append(decoded_sentence[:-1])
        print(test_text[seq_index],test_target_text[seq_index][1:-1] ,decoded_sentence[:-1])

        writer.writerow({'text': test_text[seq_index], 'truth value': test_target_text[seq_index][1:-1], 'predicted value': decoded_sentence[:-1]})


#f.close()
print(count/len(test_text))
    

ank अंक अंक
anka अंक अंका
ankit अंकित अंकीट
anakon अंकों आंकों
ankhon अंकों आंखों
ankon अंकों अंकों
angkor अंकोर अंगकोर
ankor अंकोर एंकर
angaarak अंगारक अंगरक
angarak अंगारक अंगरक
angraji अंग्रज़ी अंग्रजी
angreji अंग्रज़ी अंग्रजी
angrzi अंग्रज़ी अंग्रज्फी
antah अंतः अंतः
antaha अंतः अंतहा
antarmukh अंतर्मुख अंतरूंदु
antmurkh अंतर्मुख आंतरूढ़
antrmukh अंतर्मुख अंतरूंख
andrabee अंद्राबी अंदरबी
andrabi अंद्राबी अंदरहीदी
andha अंधा अंधा
andhapan अंधापन अंधापन
andheri अंधेरी अंधरी
ambaani अंबानी अम्बानी
ambani अंबानी अमबानी
akram अकरम अक्रम
acapulco अकापुल्को आकाफोल्कों
akapulko अकापुल्को आकापकुल
akhand अखंड अखंड
akhil अखिल अखिल
agavai अगवाई आगवाई
agvaai अगवाई अजवाई
augustine अगस्टाइन उगस्वसनी
augustustine अगस्टाइन अगुस्तसत्नी
agsta अगस्ता अजस्ट
agusta अगस्ता अजस्टु
augusta अगस्ता उगसाता
agr अग्र एग्र
agra अग्र अग्र
achambha अचंभा अनाभाष्य
achanbha अचंभा अनाख्वा
achievement अचीवमेंट अनिजीमिटम
achivement अचीवमेंट एचिबमेंट
acchi अच्छी अच्छी
achchi अच्छी अच्छी
achhi अच्छी अच्छी
achyut अच्युत अ



*  Run the below 2 cells to form a table of predicted and truth values on wandb




In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(test_text, truth_value,predicted_value)),columns =['input text', 'truth value','predicted text'])

In [ ]:

wandb.init(entity="assignment3",project ='assignment3-part1',config=configuration)  
wandb.run.name = wandb.run.name = 'hl-'+str(wandb.config.hidden_layer_size)+'-ne-'+str(wandb.config.num_encoder)+'-nd-'+str(wandb.config.num_decoder)+'-es-'+str(wandb.config.embedding_size)+'-ct-'+str(wandb.config.celltype)+'-d-'+str(wandb.config.dropout)+'-bs-'+str(wandb.config.batch_size)+'-e-'+str(50)
table = wandb.Table(dataframe=df)
wandb.log({"sample Inputs": table})

**Question 4(c)**


*   Run the below cells for the confusion matrix 




In [19]:
import csv

truth_value=[]
predicted_value =[]

decoder_target_data, encoder_input_data, decoder_input_data=encoding(test_text, test_target_text)

count =0
with tf.device('/device:GPU:0'):
  for seq_index in range(len(test_text)):

      input_seq = encoder_input_data[seq_index : seq_index +1]
      input_seq=np.tile(input_seq ,(64,1))

      
      decoded_sentence = decode_sequence(input_seq)
      if(test_target_text[seq_index][1:-1] == decoded_sentence[:-1]):
          count +=1

      #truth_value.append(test_target_text[seq_index][1:-1])
      #predicted_value.append(decoded_sentence[:-1])
      list =[]
      list[:0] = test_target_text[seq_index][1:-1]
      list =list +[' '] *(21-len(list))
      truth_value.extend(list)
      list =[]
      list[:0] = decoded_sentence[:-1]
      list =list +[' '] *(21-len(list))
      predicted_value.extend(list)
      #print(len(truth_value))
      #print(len(predicted_value))
      #break



print(count/len(test_text))

0.30608618391825854


In [29]:
target_token_index[' ']=65
truth_value = [target_token_index[char] for char in truth_value]
predicted_value = [target_token_index[char] for char in predicted_value]

In [38]:
input_tokens = ['ढ', 'ब', 'ष', 'ू', '़', 'ऊ', 'आ', 'ग', 'ख', 'ृ', 'अ', 'ु', 'त', 'ह', 'ऑ', 'ऐ', 'घ', 'ि', 'द', 'च', 'ट', 'ॉ', 'ं', 'म', 'ओ', 'ङ', 'ई', 'उ', 'ल', 'इ', 'झ', 'ॐ', 'छ', 'श', 'औ', 'भ', 'क', 'ड', 'ज', 'ा', 'ः', 'न', 'र', 'य', 'फ', 'ौ', 'ए', 'ऋ', 'ो', 'ै', 'े', 'ध', '्', 'ञ', 'थ', 'ँ', 'ण', 'व', 'प', 'ॅ', 'ठ', '\t', 'स', '\n', 'ी', ' ']


In [40]:
wandb.init(entity="assignment3",project ='assignment3-part1',config=configuration)  
wandb.run.name = "confusion matrix"

wandb.log({"confusion-matrix":wandb.plot.confusion_matrix(probs=None,
                        y_true=truth_value, preds=predicted_value,
                        class_names=input_tokens)})